
# Double drive simulator with evolutionary stability

23-02-21

Katie Willis

katie.willis16@imperial.ac.uk
___

This file contains functions required to simulate the release of a 2 locus gene drive into a single panmictic population and monitor the population through time.
We model 2 loci allowing for homing-associated loss-of-function of each transcriptional unit present in each genetic construct.
The number of alleles and genotypes will therefore depend on the contents of the constructs being released. 
We begin with a matrix containing numbers of individuals of each genotype after transgenic release, monitoring males and females separately. 
To model processes such as fitness costs, homing, sex ratio bias and recombination during each generation, we multiply (element-wise) the numbers of individuals per genotype by a series of transition matrices for each process, built using user-defined parameters. 
The output is a dictionary containing time-series of allele and genotype frequencies, numbers of indididuals and the correlation between the two gene drive constructs.
___

## Parameter and matrix definition

### Allele and genotype lists

Since the number of alleles at each locus depends on the number of components in the released construct, we first define which components are present in each construct and then calculate the number of alleles/genotypes for each design (in contrast to the [baseline](DoubleDrive_Simulator_Baseline.ipynb) and [4-allele](DoubleDrive_Simulator_4allele.ipynb) models where the number of alleles and genotypes are constant).  
Here the alleles are labelled $A_1...A_n$ at the first locus, and $B_1...B_m$ at the second locus, where $n$ and $m$ are the number of alleles at each loci.
The functions below calculate the number of alleles at each loci and generate lists of alleles and genotypes according to the contents of the constructs.

In [6]:
"""
Defines lists of alleles and genotypes as strings.
"""
function define_string_genotypes(n_alleles_locus1, #number of A alleles
                                n_alleles_locus2)  #number of B alleles
    
    A_alleles = ["A"*string(i) for i in 1:n_alleles_locus1]
    B_alleles = ["B"*string(i) for i in 1:n_alleles_locus2]
    
    #Define all A and B allele combinations: Creating single chromosomes
    alleles = []
    for A in A_alleles
        for B in B_alleles
            push!(alleles,([A,B]))
        end
    end
    
    #Define all genotypes at locus A
    A_genotypes = []
    for i in 1:length(A_alleles)
        allele1 = A_alleles[i]
        for j in i:length(A_alleles)
            allele2 = A_alleles[j]
            push!(A_genotypes,([allele1,allele2]))
        end
    end
    
    #Define all genotypes at locus B
    B_genotypes = []
    for i in 1:length(B_alleles)
        allele1 = B_alleles[i]
        for j in i:length(B_alleles)
            allele2 = B_alleles[j]
            push!(B_genotypes,([allele1,allele2]))
        end
    end
    
    #Define all genotypes for both A and B loci combined
    genotypes = []
    for i in 1:length(alleles)
        allele1 = alleles[i]
        for j in i:length(alleles)
            allele2 = alleles[j]
            push!(genotypes,([allele1 allele2]))
        end
    end
    #First index selects the genotype 
    #Second index selects the locus of interest 
    #third indec selects the chromosome of interest
    
    return(A_alleles,
            B_alleles,
            alleles,
            A_genotypes,
            B_genotypes,
            genotypes)
end

"""
Defines lists of alleles and genotypes as symbols.
"""
function define_symbolic_genotypes(n_alleles_locus1, #number of A alleles
                                    n_alleles_locus2)#number of B alleles
    A_alleles_sym = ["A"*string(i) for i in 1:n_alleles_locus1]
    B_alleles_sym = ["B"*string(i) for i in 1:n_alleles_locus2];
    
    #Define all A and B allele combinations: Creating single chromosomes
    alleles_sym = []
    for A in A_alleles_sym
        for B in B_alleles_sym
            push!(alleles_sym,(A*B))
        end
    end
    
    #Define all genotypes at locus A
    A_genotypes_sym = []
    for i in 1:length(A_alleles_sym)
        allele1 = A_alleles_sym[i]
        for j in i:length(A_alleles_sym)
            allele2 = A_alleles_sym[j]
            push!(A_genotypes_sym,(allele1*allele2))
        end
    end
    
    #Define all genotypes at locus B
    B_genotypes_sym = []
    for i in 1:length(B_alleles_sym)
        allele1 = B_alleles_sym[i]
        for j in i:length(B_alleles_sym)
            allele2 = B_alleles_sym[j]
            push!(B_genotypes_sym,(allele1*allele2))
        end
    end

    
    #Define all genotypes for both A and B loci combined
    genotypes_sym = []
    for i in 1:length(alleles_sym)
        allele1 = alleles_sym[i]
        for j in i:length(alleles_sym)
            allele2 = alleles_sym[j]
            push!(genotypes_sym,(allele1*allele2))
        end
    end
    
    A_alleles_sym = [Sym(i) for i in A_alleles_sym]
    B_alleles_sym = [Sym(i) for i in B_alleles_sym]
    alleles_sym = [Sym(a) for a in alleles_sym];
    A_genotypes_sym = [Sym(a) for a in A_genotypes_sym]
    B_genotypes_sym = [Sym(b) for b in B_genotypes_sym]
    genotypes_sym = [Sym(i) for i in genotypes_sym];

    return(alleles_sym,
            genotypes_sym)
end

define_string_genotypes

At a single locus, each allele has associated with it a number of components, or it is empty.
The alleles and their contents are defined using a dictionary.
Each dictionary key is an allele e.g. $A_1...A_n$, each value is a list of transcriptional units $∈$ {"cas9","grna","xs"}.

At a single locus, each allele also has associated with it a cost due to disruption by insertion of a construct or by a cleavage-resistant allele. Each allele is assigned a fitness type: wt = no costs; disruption = costs due to insertion; resistant = costs due to resistant mutation.
The alleles and their disruption costs are defined using a dictionary.
Each dictionary key is an allele e.g. $A_1...A_n$, each value is a string $∈$ {"wt","disruption","resistant"}.

The functions below generate the two dictionaries for each locus depending on the contents of the released constructs.

In [9]:
"""
Generates a list of the content of all possible mutant constructs
"""
function Make_construct_content_list(construct,   #List of components in construct
                                    content_list, #List of lists, each containing the contents of eahc allele
                                    L)             #number of components in the construct)
    for i in 1:length(construct)
        #removes a component of the construct
        construct_mod = deepcopy(construct)
        deleteat!(construct_mod,i)
        if length(construct_mod) == 0
            construct_mod = []
        end
        if construct_mod ∉ content_list
            push!(content_list,construct_mod)
        end
        
        #Repeat if there are still components in mutant construct 
        if length(construct_mod)<=(L-1)
            content_list = Make_construct_content_list(construct_mod,content_list,L)
        end
    end
    return(content_list)
end

"""
Generates a dictionary of all alleles, including the WT, construct, all mutant forms
and additional resistant allele if required.
"""
function Make_construct_content_dict(locus,          #A or B
                                    construct,       #List of components in construct e.g ["cas9","Agrna"]
                                    resistant_locus) #true or false for resistant locus with different fitness
    content_list = []
    L = length(construct)
    content_list = Make_construct_content_list(construct,content_list,L)
    content_list = sort(content_list,by=length,rev=true)
    
    content_dict = Dict()
    content_dict[locus*string(1)] = []
    content_dict[locus*string(2)] = construct
    
    ir_dict = Dict()
    ir_dict[locus*string(1)] = "wt"
    ir_dict[locus*string(2)] = "disruption"
    
    count = 2
    for i in 1:length(content_list)
        count = count +1
        content_dict[locus*string(count)] = content_list[i]
        ir_dict[locus*string(count)] = "disruption"
    end
    if resistant_locus == true
        count = count +1
        content_dict[locus*string(count)] = []
        ir_dict[locus*string(count)] = "resistant"
    end
    return(content_dict,ir_dict)
end

Make_construct_content_dict

### Parameters

We define a dictionary of all parameters required to run the simulation.

In [4]:
"""
Initiates a dictionary containing all parameters. 
All are set to zero except:
m = 0.5
r = 0.5
Rm = 6 
Nf_eq = 1000 (Number of females or males at equlibrium)
OJ = 1/10
therefore:
f = (2*Rm)/OJ_
α = f_*Nf_eq/(Rm-1)
"""
function Initiate_baseline_parameters()
    
    params = Dict{String,Float64}()

    #expression costs for each possible construct component
    ## for females
    params["scasf"] = 0
    params["hcasf"] = 0
    params["sAgRNAf"] = 0
    params["hAgRNAf"] = 0
    params["sBgRNAf"] = 0
    params["hBgRNAf"] = 0
    params["sXSf"] = 0
    params["hXSf"] = 0
    ## for males
    params["scasm"] = 0
    params["hcasm"] = 0
    params["sAgRNAm"] = 0
    params["hAgRNAm"] = 0
    params["sBgRNAm"] = 0
    params["hBgRNAm"] = 0
    params["sXSm"] = 0
    params["hXSm"] = 0
    
    #disruption costs
    ## for females
    params["siaf"] = 0
    params["hiaf"] = 0
    params["sraf"] = 0
    params["hraf"] = 0
    params["sibf"] = 0
    params["hibf"] = 0
    params["srbf"] = 0
    params["hrbf"] = 0
    params["siraf"] = 0
    params["sirbf"] = 0
    ## for males
    params["siam"] = 0
    params["hiam"] = 0
    params["sram"] = 0
    params["hram"] = 0
    params["sibm"] = 0
    params["hibm"] = 0
    params["srbm"] = 0
    params["hrbm"] = 0
    params["siram"] = 0
    params["sirbm"] = 0
    
    ## Activity costs
    params["sXsf"] = 0
    params["sXsm"] = 0
    params["sSaf"] = 0
    params["sSam"] = 0
    params["sSbf"] = 0
    params["sSbm"] = 0
    params["sHaf"] = 0
    params["sHam"] = 0
    params["sHbf"] = 0
    params["sHbm"] = 0
    
    # homing rates
    params["caf"] = 0
    params["jaf"] = 0
    params["cbf"] = 0
    params["jbf"] = 0
    params["cam"] = 0
    params["jam"] = 0
    params["cbm"] = 0
    params["jbm"] = 0
    
    #recombination
    params["r"] = 0.5
    
    #sex ratio bias
    params["m"] = 0.5
    
    #population dynamics 
    Rm = 6
    Nf_eq = 1000.0
    params["OJ"] = 1/10
    params["f"] = (2*Rm)/params["OJ"]
    params["α"] = params["f"]*Nf_eq/(Rm-1)
    
    #probability of NHEJ products being having the 'resistant' fitness type
    params["paf"] = 0
    params["pam"] = 0
    params["pbf"] = 0
    params["pbm"] = 0
    
    #probability of homing associated mutation of each trancriptional unit
    params["daf"] = 0
    params["dam"] = 0
    params["dbf"] = 0
    params["dbm"] = 0
    
    return(params)
end

"""
Initiates a parameter set based on our selected baseline parameters and:
    - the locus type of A and B (Ntrl, HI, HS)
    - homing dependency of A and B (Constitutive, conditional, none)
    - XS dependency (A or B)
If applied, homing rates are pre-defined according to Kyrou et al 2018
c = 0.971
j = 0.019
if applied, Xshredder rates are pre-defined according to Galizi et al 2014
m = 0.95
Fitness costs due to locus distruption are pre-defined to 1 based on the locus type
Off target cleavage costs are set to 0.01 for both males and females
"""
function Assign_baseline_params(LOCUS_A,
                                LOCUS_B,
                                set_A_homing_dependency,
                                set_B_homing_dependency,
                                set_XS_dependency)
    
    cleavage_Kyrou2018 = 0.971
    joining_Kyrou2018 = 0.019
    
    params = Initiate_baseline_parameters()
    
    if LOCUS_A == "HS_rec"
        #HS gene
        params["siaf"]=1.0 
        params["sraf"]=1.0 
        params["siraf"] = 1.0
    elseif LOCUS_A == "HI"
        #HI gene
        params["siaf"]=1.0 
        params["sraf"]=1.0 
        params["siraf"] = 1.0
        params["hiaf"]=1.0 
        params["hraf"]=1.0
    end

    if LOCUS_B == "HS_rec"
        #HS gene
        params["sibf"]=1.0 
        params["srbf"]=1.0 
        params["sirbf"] = 1.0
    elseif LOCUS_B == "HI"
        #HI gene
        params["sibf"]=1.0 
        params["srbf"]=1.0 
        params["sirbf"] = 1.0
        params["hibf"]=1.0 
        params["hrbf"]=1.0
    end

    if set_A_homing_dependency != "none"
        params["caf"] = cleavage_Kyrou2018 # homing 
        params["jaf"] = joining_Kyrou2018 # homing
        params["cam"] = cleavage_Kyrou2018 # homing 
        params["jam"] = joining_Kyrou2018 # homing
        A_homing_dependency = set_A_homing_dependency
    elseif set_A_homing_dependency == "none"
        #Doesnt matter what this is as the homing rates are set to 0.0
        A_homing_dependency = "constitutive"
    end

    if set_B_homing_dependency != "none"
        params["cbf"] = cleavage_Kyrou2018 # homing 
        params["jbf"] = joining_Kyrou2018 # homing
        params["cbm"] = cleavage_Kyrou2018 # homing 
        params["jbm"] = joining_Kyrou2018 # homing
        B_homing_dependency = set_B_homing_dependency
    elseif set_B_homing_dependency == "none"
        #Doesnt matter what this is as the homing rates are set to 0.0
        B_homing_dependency = "constitutive"
    end

    if set_XS_dependency != "none"
        params["m"] = 0.95
        XS_dependency = set_XS_dependency
    elseif set_XS_dependency == "none"
        params["m"] = 0.5
        XS_dependency = "none"
    end
    
    #off target cleavage
    params["sHaf"] = 0.01
    params["sHam"] = 0.01
    params["sHbf"] = 0.01
    params["sHbm"] = 0.01
    
    return(params)
end

Initiate_baseline_parameters (generic function with 1 method)

### Fitness

Fitness costs are calculated based on 3 different processes. The first two are locus specific, including costs due to locus disruption by either construct insertion or cleavage resistant mutation, and costs due to expression of the construct. The third is due to construct activity, which includes off-target Cas9 cleavage, somatic Cas9 cleavage and X-shredding, each of which may or may not require interaction between constructs for the cost to occur. 
Fitness costs are caculated multiplicitly: where the homozygous WT fitness cost is standardised to 1 and each process reduces the fitness by a factor.
The functions below produce a fitness matrix containing the relative fitness of each genotype compared to the wildtype depending on the contents of each construct. 
Each row of the matrix represents a genotype, the first column contains the relative fitness of females and the second the relative fitness of males for each genotype.



##### Disruption costs

Each allele has associated with it costs due to disruption of the locus.
The fitness due to disruption of function is as follows: 

\begin{align}
{W/W} & = 1 \\
{W/D} & = 1-s_Ih_I \\
{D/D} & = 1-s_I \\
{W/R} & = 1-s_Rh_R \\
{R/R} & = 1-s_R \\
{D/R} & = 1-s_{IR} \\
\end{align}

Where $s_I$ and $h_I$ are the selection and domcinance coefficient for disruption by insertion of the construct or any of its mutant forms (D), $s_R$ and $h_R$ are the selection and dominance coefficient for disruption by a resistance allele (R) and $s_{IR}$ is the selection coefficient for the heterozygotes carrying either a construct or a mutant construct and a cleavage resistant allele.
Each parameter differs depending on locus and sex. 


In [1]:
"""
Makes the fitness matrix of female and male individuals taking into consideration disruption of 
function at a single locus.
The output is a matrix, where each column represents the sex (female, male)
and each row represents the genotype of a diploid individuals at a single locus.
"""
function Get_disruption_fitness(alleles_ir,locus_genotypes,
                                sif::Float64,hif::Float64,srf::Float64,hrf::Float64,sirf::Float64,
                                sim::Float64,him::Float64,srm::Float64,hrm::Float64,sirm::Float64)
    
    #Define dictionaries for disruption/insertion
    component_ir_dict_f = Dict([("wt", [0,0]), ("disruption", [sif, hif]), ("resistant", [srf, hrf])]);
    component_ir_dict_m = Dict([("wt", [0,0]), ("disruption", [sim, him]), ("resistant", [srm, hrm])]);
    
    #storage
    fitness_f_vec = []
    fitness_m_vec = []
    
    for genotype in locus_genotypes

        #Collect info on alleles
        ir_type_1 = alleles_ir[genotype[1]]
        ir_type_2 = alleles_ir[genotype[2]]
        #Assign fitness due to interuption of the gene
        ##fitness costs of 2 components with the same insertion effect
        if ir_type_1 == ir_type_2
            fitness_f = (1-component_ir_dict_f[ir_type_1][1])
            fitness_m = (1-component_ir_dict_m[ir_type_1][1])
        ##fitness costs of 2 components with different insertion effects
        ###if one is the WT we use heterozygote parameters
        elseif (ir_type_1 == "wt") & (ir_type_2 != "wt")
            fitness_f = (1-(component_ir_dict_f[ir_type_2][1]*component_ir_dict_f[ir_type_2][2]))
            fitness_m = (1-(component_ir_dict_m[ir_type_2][1]*component_ir_dict_m[ir_type_2][2]))
            #println(genotype,fitness_f,fitness_m)
        elseif (ir_type_1 != "wt") & (ir_type_2 == "wt")
            fitness_f = (1-(component_ir_dict_f[ir_type_1][1]*component_ir_dict_f[ir_type_1][2]))
            fitness_m = (1-(component_ir_dict_m[ir_type_1][1]*component_ir_dict_m[ir_type_1][2]))
            #println(genotype,fitness_f,fitness_m)
        elseif (ir_type_1 != "wt") & (ir_type_2 != "wt")
            fitness_f = (1-sirf)
            fitness_m = (1-sirm)
        else 
            print("missed an option")
        end
        push!(fitness_f_vec,fitness_f)
        push!(fitness_m_vec,fitness_m)
    end
    return(hcat(fitness_f_vec,fitness_m_vec))
end

Get_disruption_fitness


##### Expression costs

Each allele may or may not contain transcriptional units, each of which has expression costs associated. 
The fitness due to expression of these components is as follows: 

\begin{align}
{Hom} & = \prod_{k=1}^n (1-s_k)\\
{Het} & = \prod_{k=1}^n (1-s_kh_k) + \prod_{l=1}^n (1-s_lh_l) \\
\end{align}

where k refers to fitness effects associated with each component present in the allele, if heterozygous k and l refer to fitness effects associated with each component present in the first and second allele reprectively.



In [ ]:
"""
Makes the fitness matrix of female and male individuals considering the contents of the construct at a single locus..
The output is a matrix, where each column rperesents the sex (female, male)
and each row represents the genotype of a diploid individuals at a single locus.
"""
function Get_construct_expression_fitness(alleles_content,locus_genotypes,p)

    component_fitness_dict = Dict([("cas9", [p["scasf"],p["hcasf"],p["scasm"],p["hcasm"]]), 
            ("Agrna", [p["sAgRNAf"],p["hAgRNAf"],p["sAgRNAm"],p["hAgRNAm"]]), 
            ("Bgrna", [p["sBgRNAf"],p["hBgRNAf"],p["sBgRNAm"],p["hBgRNAm"]]), 
            ("xs", [p["sXSf"],p["hXSf"],p["sXSm"],p["hXSm"]])])
    
    fitness_f_vec = Vector{Float64}()
    fitness_m_vec = Vector{Float64}()
    
    for genotype in locus_genotypes
    
        #full fitness to begin with
        fitness_f = 1
        fitness_m = 1
        
        #Collect info on alleles
        content_1 = alleles_content[genotype[1]]
        content_2 = alleles_content[genotype[2]]
        
        #Assign fitness due to cost of expressing content of construct
        ##Homozygote fitness costs of construct expression 
        if genotype[1] == genotype[2]
            
            #fitness due to expression of component contents
            for component in content_1
                fitness_f = fitness_f*(1-component_fitness_dict[component][1])
                fitness_m = fitness_m*(1-component_fitness_dict[component][3])
            end

        ##Heterozygous fitness costs of construct expression 
        elseif genotype[1] != genotype[2]
            for component in content_1
                fitness_f = fitness_f*(1-(component_fitness_dict[component][1]*component_fitness_dict[component][2]))
                fitness_m = fitness_m*(1-(component_fitness_dict[component][3]*component_fitness_dict[component][4]))
            end
            for component in content_2
                fitness_f = fitness_f*(1-(component_fitness_dict[component][1]*component_fitness_dict[component][2]))
                fitness_m = fitness_m*(1-(component_fitness_dict[component][3]*component_fitness_dict[component][4]))
            end
        end
        push!(fitness_f_vec,fitness_f)
        push!(fitness_m_vec,fitness_m)
    end
    return(hcat(fitness_f_vec,fitness_m_vec))
end


The fitness of an individual based on the diploid genotype at a single allele is calculated by multplying the fitness due to expression of the components by the fitness due to disruption of function at the locus.
The fitness for an individual considering expression and locus disruption at both loci is calculated by multiplying the fitness costs based on each single locus.

In [ ]:
"""
Combines the A and B 1-locus fitness matrices to generate a matrix where 
rows represent each 2-locus diploid genotype, for female (column 1) and male (column 2) fitness costs for locus 
disruption and construct expression. 
"""
function Get_expression_fitness_matrix(A_fitness_matrix,B_fitness_matrix,A_genotypes,B_genotypes,genotypes)
    
    #Calculate the fitness of each combined genotype by multiplying the
    # fitness for genotypes at each locus.

    fitness_M_store = [] #male fitness
    fitness_F_store = [] #female fitness

    #For every combined genotype e.g. ABAB
    for genotype in genotypes
        
        B_fitness = []
        A_fitness = []
        
        #Check what the B genotype is e.g. BB
        #and assign B_fitness accordingly:
        for i in 1:length(B_genotypes) #iterate through the B genotypes
            B_genotype = B_genotypes[i] #Split to get the two alleles
            if ( #Check if both alleles occur in the combined genotype (in any order)
                    ((B_genotype[1]==genotype[2,1]) & (B_genotype[2]==genotype[2,2]))
                    |
                    ((B_genotype[2]==genotype[2,1]) & (B_genotype[1]==genotype[2,2]))
                )
                #If they do assign the fitness
                B_fitness = B_fitness_matrix[i,:]
            end
        end

        #Check what the A genotype is e.g. AA
        #and assign A_fitness accordingly:
        for i in 1:length(A_genotypes) 
            A_genotype = A_genotypes[i] #iterate through the A genotypes
            if ( #Check if both alleles occur in the combined genotype (in any order)
                    ((A_genotype[1]==genotype[1,1]) & (A_genotype[2]==genotype[1,2]))
                    |
                    ((A_genotype[2]==genotype[1,1]) & (A_genotype[1]==genotype[1,2]))
                )
                #If they do assign the fitness
                A_fitness = A_fitness_matrix[i,:]
            end
        end

        #Multiply the fitnesses at A and B (Assume no epistatic effects)
        combined_fitness = simplify.(A_fitness .* B_fitness)
        fitness_F = combined_fitness[1]
        fitness_M = combined_fitness[2]
        #Store male and female fitnesses independently
        push!(fitness_M_store,fitness_M)   
        push!(fitness_F_store,fitness_F)   
    end
    
    #Combine male and female fitnesses for each combined genotype into a matrix
    return(hcat(fitness_F_store,fitness_M_store))
end


##### Activity costs

Off-target cleavage reduces fitness by a factor $1 - s_{H}$ in the presence of Cas9 and gRNA and $(1 - s_{H})^2$ in the presence of Cas9 and two different gRNAs (gRNA targeting A and gRNA targeting B). 
Somatic cleavage reduces fitness by a factor $1-s_{S}$ in the presence of Cas9, gRNA and the target site of gRNA. 
X-shredding reduces fitness by a factor $1-s_{XS}$ in the presence of XS.
Each selection coefficient can vary depending on the sex. 
Here we assume the costs associated with each activity are dominant, i.e. the presence of 1 unit results in the same fitness cost as 2 units.

The final fitness for each genotype is calcuated by multiplying the per genotype fitness based on expression and locus disruption by the fitness due to synergistic effects. 

In [ ]:
"""
Takes in the lists of construct content and list of genotypes and calculates 
the relative fitness due to interactions between components of constructs.
Outputs a fitness matrix consisting of two columns (females and males),
where each row is a 2-locus diploid genotype containing relative fitness due to construct activity.
"""
function Get_synergistic_fitness(A_alleles_content,B_alleles_content,genotypes,p)
    
    fitness_M_store = Vector{Float64}() #male fitness
    fitness_F_store = Vector{Float64}() #female fitness
    
    for i in 1:length(genotypes)
        
        genotype_1 = genotypes[i][:,1]
        genotype_2 = genotypes[i][:,2]
        genotype_1_Acontent = A_alleles_content[genotype_1[1]]
        genotype_1_Bcontent = B_alleles_content[genotype_1[2]]
        genotype_2_Acontent = A_alleles_content[genotype_2[1]]
        genotype_2_Bcontent = B_alleles_content[genotype_2[2]]
        
        fitness_m = 1.0
        fitness_f = 1.0
        
        ## Off target cleavage and somatic leakage
        #if there is at least one Cas9...
        if (("cas9" in genotype_1_Acontent) | ("cas9" in genotype_2_Acontent)) | (("cas9" in genotype_1_Bcontent) | ("cas9" in genotype_2_Bcontent))
            
            #and at least one grna-A for off target cleavage
            if (("Agrna" in genotype_1_Acontent) | ("Agrna" in genotype_2_Acontent)) | (("Agrna" in genotype_1_Bcontent) | ("Agrna" in genotype_2_Bcontent))
                fitness_m = fitness_m*(1-p["sHam"])
                fitness_f = fitness_f*(1-p["sHaf"])
                #and at least one A target site for somatic leakage
                if ("A1" in genotype_1) |("A1" in genotype_2)
                    fitness_m = fitness_m*(1-p["sSam"])
                    fitness_f = fitness_f*(1-p["sSaf"])
                end
            end
    
            #and at least one grna-B for off target cleavage
            if (("Bgrna" in genotype_1_Acontent) | ("Bgrna" in genotype_2_Acontent)) | (("Bgrna" in genotype_1_Bcontent) | ("Bgrna" in genotype_2_Bcontent))
                fitness_m = fitness_m*(1-p["sHbm"])
                fitness_f = fitness_f*(1-p["sHbf"])
                #and at least one B target site for somatic leakage
                if ("B1" in genotype_1) | ("B1" in genotype_2)
                    fitness_m = fitness_m*(1-p["sSbm"])
                    fitness_f = fitness_f*(1-p["sSbf"])
                end
            end
        end
    
        ## X-shredder cost
        if (("xs" in genotype_1_Acontent) | ("xs" in genotype_2_Acontent)) | (("xs" in genotype_1_Bcontent) | ("xs" in genotype_2_Bcontent))
            fitness_m = fitness_m*(1-p["sXsf"])
            fitness_f = fitness_f*(1-p["sXsm"])
        end

        push!(fitness_M_store,fitness_m)   
        push!(fitness_F_store,fitness_f)   
        
    end
    return(hcat(fitness_F_store,fitness_M_store))
end

In [12]:
"""
Wrapper: 
Input: 
* 2 lists of construct components, one for each construct A and B
* 2 lists of construct component fitness types, one for each construct A and B
* 2 lists of diploid alleles at a single locus, one for each construct A and B
* 1 list of 2-locus diploid genotypes

The function calculates the fitness for each 2-locus diploid genotype 
e.g. ABAB, taking into consideration costs due to 
1) locus disruption 
2) construct expression 
3) construct activity 
Outputs a fitness matrix containing relative fitnesses of females and males (columns) for each
2-locus diploid genotype (row). 
"""
function Build_fitness_matrix(A_alleles_content,A_alleles_ir,A_genotypes,B_alleles_content,B_alleles_ir,B_genotypes,genotypes,p)
    
    A_fitness_matrix = Get_disruption_fitness(A_alleles_ir,A_genotypes,
                                p["siaf"],p["hiaf"],p["sraf"],p["hraf"],p["siraf"],
                                p["siam"],p["hiam"],p["sram"],p["hram"],p["siram"]) .* 
                        Get_construct_expression_fitness(A_alleles_content,A_genotypes,p)
    
    B_fitness_matrix = Get_disruption_fitness(B_alleles_ir,B_genotypes,
                                p["sibf"],p["hibf"],p["srbf"],p["hrbf"],p["sirbf"],
                                p["sibm"],p["hibm"],p["srbm"],p["hrbm"],p["sirbm"]) .* 
                        Get_construct_expression_fitness(B_alleles_content,B_genotypes,p)
    
    disruption_expression_matrix = Get_expression_fitness_matrix(A_fitness_matrix, B_fitness_matrix,A_genotypes,B_genotypes,genotypes)
    synergistic_matrix = Get_synergistic_fitness(A_alleles_content,B_alleles_content,genotypes,p)
    
    fitness_matrix = disruption_expression_matrix .* synergistic_matrix
    
    return(fitness_matrix)
end

Get_synergistic_fitness

### Homing

Homing in the germline acts to convert genotypes from one to another before segregation during meiosis.
Homing requires the presence of Cas9, gRNA and a target alelle and involves two processes. 
The first is cleavage, which occurs with probability $c$. 
The second is repair which can either occur via NHEJ with probability $j$, converting the WT to a resistant allale, or HDR with probability $1-j$.
If NHEJ occurs, the WT is converted into an allele containing no transcripitonal units with the "resistant" fitness type with probability $p$ or with the "insertion" fitness type with probability $1-p$, allowing for modelling of both functional and non-functional resistance.
To study evolutionary stability, if HDR has occured each transcriptional unit of each construct has a probability $d$ of mutating to a non-functional version, whilst maintaining fitness costs due to locus disruption (the unit is mutated rather than lost).
The functions below generate a genotype x genotype (i x j) matrix, which contain the proportion of each pre-homed genotype (i, row) converted to each post-homed genotypes (j, column) due to homing.


In [13]:
"""
Sorts the allele string so order is in numerical order, where A > B
"""
function sort_allele(unsorted_allele)
    n1 = parse(Int64,unsorted_allele[1][2:end])
    n2 = parse(Int64,unsorted_allele[2][2:end])
    if n1 > n2
        sorted_allele = [unsorted_allele[2,:]; unsorted_allele[1,:]]
    else
        sorted_allele = [unsorted_allele[1,:]; unsorted_allele[2,:]]
    end
    return(sorted_allele)
end

"""
Sorts the genotype string so order is in numerical order, where A > B
"""
function sort_genotype(unsorted_genotype)
    sorted_genotype = ["X" "Y"; "X" "Y"]
    if unsorted_genotype[1,1] != unsorted_genotype[1,2]
        if sort_allele(unsorted_genotype[1,:]) == unsorted_genotype[1,:]
            sorted_genotype = unsorted_genotype
        else
            sorted_genotype[:,1] = unsorted_genotype[:,2]
            sorted_genotype[:,2] = unsorted_genotype[:,1]
        end
    else
        if sort_allele(unsorted_genotype[2,:]) == unsorted_genotype[2,:]
            sorted_genotype =unsorted_genotype
        else
            sorted_genotype[:,1] = unsorted_genotype[:,2]
            sorted_genotype[:,2] = unsorted_genotype[:,1]
        end
    end
    return(sorted_genotype)
end


sort_genotype

In [14]:
"""
Wrapper to calculate the construct dictionary for a specific construct
"""
function get_transition_dict(alleles_ir,
                                    construct_label,
                                    content_dict,
                                    c,j,d,p)
    
    construct_alleles = [k for (k,v) in alleles_ir if v=="disruption"] #list of labels of construct + derivatives
    construct_content = content_dict[construct_label] #focal construct content
    transition_dict = Build_transition_dict(construct_content,c,j,d,p) #transition dictionary for focal construct
    transition_dict = relabel_transition_dict(transition_dict,content_dict,construct_alleles)
    return(transition_dict)
end

"""
Takes in a list of construct components 
Calculates all possible mutant variations of the construct 
Calculates the equations which represent the probability of each mutant construct being produced.
Returns a dictionary where the key is the list of construct components and the value is the equation.
"""
function Build_transition_dict(construct,c,j,d,p)
    #Save the length of the intact construct
    L = length(construct)
    #Initiate the construct dictionary
    transition_dict = Dict()
    #IF there are components present in the construct mutation can occur
    if L > 0
        #Calculate the probability of producing each mutant construct
        transition_dict = Make_transition_equation_dict(construct,transition_dict,c,j,d,L)
        #add in the probability a non-functional mutant can be produced through end-joining 
        ## rather than mutation
        transition_dict[[]] = transition_dict[[]]+c*j*(1-p)
        #add in the probability of homing of the intact construct
        n=0
        transition_dict[construct] = c*(1-j)*d^n*(1-d)^(L-n)

    #IF there are no components in the construct, no mutation to a non-functional resistant allele occurs
    else
        #add in the probability the mutant being produced through end-joining
        #add in the probability the mutant is produced through homing of the original disruption
        transition_dict[[]] = c*j*(1-p)+c*(1-j)
    end
    return(transition_dict)
end

"""
Takes in a construct, its length and a dictionary where the key is the list of construct components 
and the value is the equation.
L = number of transcriptional units in the construct
n = number of transcriptional units in the broken construct
"""
function Make_transition_equation_dict(construct,transition_dict,c,j,d,L)

    #iterates through the length of the construct
    for i in 1:length(construct)
        #removes a component of the construct
        construct_mod = deepcopy(construct)
        deleteat!(construct_mod,i)
        
        #calculates the length of the mutant construct
        n= L-length(construct_mod)
        
        #subs the values into the equation
        #c = probability of cleavage
        #1-j = probability of homing given cleavage
        #d = probability a single transcriptional unit mutates per homing event
        #L = number of transcriptional units in the construct
        #n = number of transcriptional units in the broken construct
        equation_vals = c*(1-j)*d^n*(1-d)^(L-n)
        
        #Assign equation values to the dictionary
        if construct_mod in keys(transition_dict)
            transition_dict[construct_mod] = transition_dict[construct_mod]
        else
            transition_dict[construct_mod] = equation_vals
        end
        
        #Repeat if there are still components in mutant construct 
        if length(construct_mod)<=(L-1)
            transition_dict = Make_transition_equation_dict(construct_mod,transition_dict,c,j,d,L)
        end
    end
    return(transition_dict)
end

"""
Takes in construct dictionary, content dictionary and construct alleles list and relabels the dictionary
with construct labels rather than content.
"""
function relabel_transition_dict(transition_dict,content_dict,construct_alleles)
    relabelled_transition_dict = Dict()
    for label in construct_alleles
        content = content_dict[label]
        if content in keys(transition_dict)
            equation = transition_dict[content]
            relabelled_transition_dict[label] = equation
        end
    end
    return(relabelled_transition_dict)
end

relabel_transition_dict

In [15]:
"""
Fills a row of the homing matrix with the probabily that the focal genotype (row) will be
converted to each other genotype (column) when cleavage occurs only at 1 locus (A OR B).
"""
function fill_homing_matrix(homing,genotype,genotypes,index,alleles_ir,res_allele,construct_dict,modified_locus,modified_chrom,c,j,p)
    #no change
    homing[index,index] = [1-c]
    #If functional resistance is possible:
    if "resistant" in values(alleles_ir)
        #mutating into a cleavage resistant mutant with different fitness to the disrupted site
        mutated_genotype = deepcopy(genotype)
        mutated_genotype[modified_locus,modified_chrom] = res_allele 
        mutated_genotype = sort_genotype(mutated_genotype)
        homing[index,findall(x->x==mutated_genotype,genotypes)] = [c*j*p]
    end
    #For every cons
    for construct in keys(construct_dict)#1:length(construct_alleles)
        homed_mut_genotype = deepcopy(genotype)
        homed_mut_genotype[modified_locus,modified_chrom] = construct
        homed_mut_genotype = sort_genotype(homed_mut_genotype)
        homing[index,findall(x->x==homed_mut_genotype,genotypes)] = [construct_dict[construct]]
    end
    return(homing)
end
"""
Fills a row of the homing matrix with the probabily that the focal genotype (row) will be
converted to each other genotype (column) when cleavage occurs at the B locus and 
has already occured at the A locus.
"""
function fill_homing_matrix_BafterA(homing,index,genotypes,ir_dict,content_dict,a_genotype,a_equation,resB_allele,B_alleles_ir,cb,jb,db,pb)  

    modg1a,modg1b,modg2a,modg2b = a_genotype
    if ir_dict[modg1b] == "wt"
        modified_Bchrom = 1
        Bconstruct_dict = get_transition_dict(B_alleles_ir,
                                            modg2b,
                                            content_dict,
                                            cb,jb,db,pb)  
    elseif ir_dict[modg2b] == "wt"
        modified_Bchrom = 2
        Bconstruct_dict = get_transition_dict(B_alleles_ir,
                                            modg1b,
                                            content_dict,
                                            cb,jb,db,pb)
    end

    #No change to genotype_b genotype
    ab_genotype = deepcopy(a_genotype)
    ab_equation = a_equation*(1-cb)
    homing[index,findall(x->x==ab_genotype,genotypes)] = [ab_equation]

    if "resistant" in values(B_alleles_ir)
        #cleavage resistant functional mutant
        ab_genotype = deepcopy(a_genotype)
        ab_genotype[2,modified_Bchrom] = resB_allele
        ab_genotype = sort_genotype(ab_genotype)
        ab_equation = a_equation*cb*jb*pb
        homing[index,findall(x->x==ab_genotype,genotypes)] = [ab_equation]
    end
    #construct genotypes
    for Bconstruct in keys(Bconstruct_dict)
        ab_genotype = deepcopy(a_genotype)
        ab_genotype[2,modified_Bchrom] = Bconstruct
        ab_genotype = sort_genotype(ab_genotype)
        homing[index,findall(x->x==ab_genotype,genotypes)] = [a_equation*Bconstruct_dict[Bconstruct]]
    end
    return(homing)
end

"""
Iterates through all genotypes and fills the homing matrix in with transition probabilities from genotypes prior
to homing (rows) to genotypes after homing (columns). 
"""
function Build_homing_matrix(A_alleles,A_alleles_content,A_alleles_ir,B_alleles,B_alleles_content,B_alleles_ir,genotypes,ca,cb,ja,jb,pa,pb,da,db)

    #Generate content and insertion dictionaries for each allele
    content_dict = merge(A_alleles_content,B_alleles_content)
    ir_dict = merge(A_alleles_ir,B_alleles_ir)
    
    #find allele label for functional resistant loci is present
    if "resistant" in values(A_alleles_ir)
        pa=pa
        resA_allele = findall(x->x=="resistant",A_alleles_ir)[1]
    else
        resA_allele = " "
        pa=0.0
    end      
    
    if "resistant" in values(B_alleles_ir)
        pb=pb
        resB_allele = findall(x->x=="resistant",B_alleles_ir)[1]
    else
        resB_allele = " "
        pb=0.0
    end

    #Initiate the homing construct
    homing = Array{Float64}(undef,length(genotypes),length(genotypes))
    fill!(homing, 0.0)

    #We begin by assuming no homing, therefore all diagonals are set to 1.0
        #This is just so we dont have to assign all genotypes where no homing occurs.
    homing[diagind(homing, 0)] .= 1.0

    count = 0
    for genotype in genotypes

        count = count+1
        index = findall(x->x==genotype,genotypes) 
        
        #Store the allele labels for each allele at A and B sites
        g1a,g1b,g2a,g2b = genotype
        #Get the transition probability dictionaries for each allele depending on its content
        g1a_construct_dict = get_transition_dict(A_alleles_ir,
                                                        g1a,
                                                        content_dict,
                                                        ca,ja,da,pa)
        g2a_construct_dict = get_transition_dict(A_alleles_ir,
                                                        g2a,
                                                        content_dict,
                                                        ca,ja,da,pa)
        g1b_construct_dict = get_transition_dict(B_alleles_ir,
                                                        g1b,
                                                        content_dict,
                                                        cb,jb,db,pb)
        g2b_construct_dict = get_transition_dict(B_alleles_ir,
                                                        g2b,
                                                        content_dict,
                                                        cb,jb,db,pb)  
        
        #IS THERE A CAS9 present at least once:  
        if (("cas9" in content_dict[g1a]) | ("cas9" in content_dict[g2a]) | ("cas9" in content_dict[g1b]) | ("cas9" in content_dict[g2b]))
            
            #Cleavage is possible at locus A only
                #gRNA-A is present at least once
            if ((("Agrna" in content_dict[g1a]) | ("Agrna" in content_dict[g2a]) | ("Agrna" in content_dict[g1b]) | ("Agrna" in content_dict[g2b]))
                #AND gRNA-B is absent OR B-WT is absent OR B-WT is present twice
                & ((("Bgrna" ∉ content_dict[g1a]) & ("Bgrna" ∉ content_dict[g2a]) & ("Bgrna" ∉ content_dict[g1b]) & ("Bgrna" ∉ content_dict[g2b]))
                | ((ir_dict[g1b]!="wt") & (ir_dict[g2b]!="wt")) 
                        | ((ir_dict[g1b]=="wt") & (ir_dict[g2b]=="wt")) ))
                
                 modified_locus = 1 #editing occurs only at the A locus
            
                #Check whether there is a target site at A
                #2.1 #if there is one WT, homing will occur. 
                if ((ir_dict[g1a]=="wt") & ((ir_dict[g2a]=="disruption")|(ir_dict[g2a]=="resistant"))) 
                    modified_chrom = 1
                    construct_dict = g2a_construct_dict
                    homing = fill_homing_matrix(homing,genotype,genotypes,index,A_alleles_ir,resA_allele,construct_dict,modified_locus,modified_chrom,ca,ja,pa)   
                #2.2 #if there is one WT, homing will occur. 
                elseif ((ir_dict[g2a]=="wt") & ((ir_dict[g1a]=="disruption")|(ir_dict[g1a]=="resistant")))   
                    modified_chrom = 2
                    construct_dict = g1a_construct_dict
                    homing = fill_homing_matrix(homing,genotype,genotypes,index,A_alleles_ir,resA_allele,construct_dict,modified_locus,modified_chrom,ca,ja,pa) 
                #2.3 #if there are 2 wt, mutation will occur.
                elseif ((ir_dict[g1a]=="wt") & (ir_dict[g2a]=="wt"))
                    #TO BE COMLPETED.
                #2 end
                end
            

                
            #Cleavage is possible at locus B only
                #gRNA-B is present at least once
            elseif ((("Bgrna" in content_dict[g1a]) | ("Bgrna" in content_dict[g2a]) | ("Bgrna" in content_dict[g1b]) | ("Bgrna" in content_dict[g2b]))
                #AND gRNA-A is absent OR A-WT is absent OR A-WT is present twice
                & ((("Agrna" ∉ content_dict[g1a]) & ("Agrna" ∉ content_dict[g2a]) & ("Agrna" ∉ content_dict[g1b]) & ("Agrna" ∉ content_dict[g2b]))
                | ((ir_dict[g1a]!="wt") & (ir_dict[g2a]!="wt"))
                        | ((ir_dict[g1a]=="wt") & (ir_dict[g2a]=="wt"))))
                
                modified_locus = 2 #editing occurs only at the A locus
            
                #Check whether there is a target site 
                #3.1 #if there is one WT, homing will occur. 
                if ((ir_dict[g1b]=="wt") & ((ir_dict[g2b]=="disruption")|(ir_dict[g2b]=="resistant")))
                    modified_chrom = 1
                    construct_dict = g2b_construct_dict
                    homing = fill_homing_matrix(homing,genotype,genotypes,index,B_alleles_ir,resB_allele,construct_dict,modified_locus,modified_chrom,cb,jb,pb)
                #3.2 #if there is one WT, homing will occur. 
                elseif ((ir_dict[g2b]=="wt") & ((ir_dict[g1b]=="disruption")|(ir_dict[g1b]=="resistant")))
                    modified_chrom = 2
                    construct_dict = g1b_construct_dict
                    homing = fill_homing_matrix(homing,genotype,genotypes,index,B_alleles_ir,resB_allele,construct_dict,modified_locus,modified_chrom,cb,jb,pb)
                #3.3 #if there are 2 wt, mutation will occur.
                elseif ((ir_dict[g1b]=="wt") & (ir_dict[g2b]=="wt"))
                    #TO BE COMLPETED.
                #3 end
                end
            #Cleavage is possible at A and B loci
                #both gRNA-A and gRNA-B are present at least once
            elseif ((("Agrna" in content_dict[g1a]) | ("Agrna" in content_dict[g2a]) | ("Agrna" in content_dict[g1b]) | ("Agrna" in content_dict[g2b]))
                &
                (("Bgrna" in content_dict[g1a]) | ("Bgrna" in content_dict[g2a]) | ("Bgrna" in content_dict[g1b]) | ("Bgrna" in content_dict[g2b])))

                # if there is one WT at A and 1 WT at B = homing possible at both 
                if ((((ir_dict[g1a]=="wt") & ((ir_dict[g2a]=="disruption")|(ir_dict[g2a]=="resistant"))) 
                   | ((ir_dict[g2a]=="wt") & ((ir_dict[g1a]=="disruption")|(ir_dict[g1a]=="resistant"))))
                   &
                   (((ir_dict[g1b]=="wt") & ((ir_dict[g2b]=="disruption")|(ir_dict[g2b]=="resistant"))) 
                   | ((ir_dict[g2b]=="wt") & ((ir_dict[g1b]=="disruption")|(ir_dict[g1b]=="resistant"))))
                    )
                    
                    #Define which are the donor chromosomes and which are the wt (modified)
                    if ir_dict[g1a] == "wt"
                        modified_Achrom = 1
                        Aconstruct_dict = g2a_construct_dict
                    elseif ir_dict[g2a] == "wt"
                        modified_Achrom = 2
                        Aconstruct_dict = g1a_construct_dict
                    end 
                    
                    #Double homing
                    a_genotype = deepcopy(genotype)
                    a_equation = 1-ca
                    homing = fill_homing_matrix_BafterA(homing,index,genotypes,ir_dict,content_dict,a_genotype,a_equation,resB_allele,B_alleles_ir,cb,jb,db,pb)
                    
                    #attempted home and change into functional resistant mutant
                    if "resistant" in values(A_alleles_ir)
                        #cleavage resistant functional mutant
                        a_genotype = deepcopy(genotype)
                        a_genotype[1,modified_Achrom] = resA_allele
                        a_genotype = sort_genotype(a_genotype)
                        a_equation = ca*ja*pa
                        homing = fill_homing_matrix_BafterA(homing,index,genotypes,ir_dict,content_dict,a_genotype,a_equation,resB_allele,B_alleles_ir,cb,jb,db,pb)
                    end
                    #attempted home and change into mutant construct 
                    for Aconstruct in keys(Aconstruct_dict)
                        a_genotype = deepcopy(genotype)
                        a_genotype[1,modified_Achrom] = Aconstruct
                        a_genotype = sort_genotype(a_genotype)
                        a_equation = Aconstruct_dict[Aconstruct]
                        homing = fill_homing_matrix_BafterA(homing,index,genotypes,ir_dict,content_dict,a_genotype,a_equation,resB_allele,B_alleles_ir,cb,jb,db,pb)
                    end
                end
            end
        end
    end
    return(homing)                                                                                                                                                                                                
end


Build_homing_matrix

### Sex bias

The probability of a male producing an X- or Y-bearing gamete depends on the presence of an X-shredder. The functions below generate a matrix containing the proportion of X- (columns 1) or Y- (column 2) bearing sperm depending on the genotype (row). m is the proporion of Y-bearing sperm produced for a male carrying at least one copy of the X-shredder. We assume that x-shredding function is dominant, thus heterozygotes and homozygotes for the xshredder create the same sex bias. 

In [ ]:
"""
sex distortion is dependent on the the presence of an "xs" in the constuct
"""
function Build_sex_ratio_matrix(A_alleles,A_alleles_content,B_alleles,B_alleles_content,genotypes,p)
    
    m = p["m"]
    content_dict = merge(A_alleles_content,B_alleles_content)
    #Calculate the sex ratio of each genotype
    sex_ratio_storeM = Vector{Float64}()
    sex_ratio_storeF = Vector{Float64}()
    #For each combined genotype e.g. ABAB
    for genotype in genotypes
        sex_ratio = []
        #Check if each the genotype carries an X shredder and assign sex ratio accordingly:
        if (("xs" in content_dict[genotype[1,1]])|("xs" in content_dict[genotype[1,2]])|("xs" in content_dict[genotype[2,1]])|("xs" in content_dict[genotype[2,2]]))
            sex_ratio = m #WT 
            push!(sex_ratio_storeF,1-sex_ratio)   
            push!(sex_ratio_storeM,sex_ratio) 
        else
            push!(sex_ratio_storeF,0.5)   
            push!(sex_ratio_storeM,0.5) 
        end
    end
    sex_ratio_matrix = hcat(sex_ratio_storeF,sex_ratio_storeM);
    return(sex_ratio_matrix)
end


### Recombination

Recombination can occur between the two loci with probability r during gamete production. 
The function generates a matrix of the proportion of each gamete (column) produced from each genotype (row), 
where each row refers to a diploid genotype (individual) and each column to a haploid genotype (gamete) at the autosomal loci only.


In [3]:
"""
Calculate the probability of each genotype producing each gamete (allele)
taking into consideration the recombination probability r between A and B loci.
Returns matrix of the proportion of each gamete (columns) produced by each parental genotype (row)
"""
function Build_recombination_matrix(genotypes,alleles,p::Dict{String,Float64})
    r = p["r"]
    
    recombination_matrix = Array{Float64}(undef,(length(genotypes),length(alleles)))

    empty_row = [0.0 for i in 1:length(alleles)]
    empty_row = reshape(empty_row, 1, length(empty_row))

    for a in enumerate(genotypes)
        local genotype = a[2]
        local i = a[1]
        
        #for genotype in genotypes

        recombination_vec = deepcopy(empty_row)

        #Screen whether each allele (gamete) is present in each genotype
        nonrecombinant_mask = [(a == genotype[:,1])|(a == genotype[:,2]) for a in alleles]
        #if 2 alleles are present (the genotype is heterozygous)

        if sum(nonrecombinant_mask) == 2
            #Assign the 2 matched alleles non-recombinant inheritence (1-r)/2
            recombination_vec[nonrecombinant_mask] = collect([(1-r)/2, (1-r)/2])
            #Assign the rest of the gametes that are not produced and inheritance of 0
            recombination_vec[nonrecombinant_mask .== false] = empty_row[:,1:end-2]
        #if 1 allele is present (the genotype is homozygous)
        elseif sum(nonrecombinant_mask) == 1
            #Assign the 1 matched allele non-recombinant inheritance (1-r)
            recombination_vec[nonrecombinant_mask] = collect([(1-r)])
            #Assign the rest of the gametes that are not produced and inheritance of 0
            recombination_vec[nonrecombinant_mask .== false] = empty_row[:,1:end-1]
        end

        #Recombine the genotype
        genotype_recomb = ["X" "Y";"X" "Y"]
        genotype_recomb[1,1] = genotype[1,1]
        genotype_recomb[1,2] = genotype[1,2]
        genotype_recomb[2,1] = genotype[2,2]
        genotype_recomb[2,2] = genotype[2,1]

        #Screen whether each allele (gamete) is present in each genotype
        recombinant_mask = [(a == genotype_recomb[:,1])|(a == genotype_recomb[:,2]) for a in alleles]
        #if 2 alleles are present (the genotype is heterozygous)
        if sum(recombinant_mask) == 2
            #Assign the 2 matched alleles recombinant inheritence r/2
            recombination_vec[recombinant_mask] = collect(recombination_vec[recombinant_mask]+[r/2,r/2])
            #if 1 allele is present (the genotype is homozygous)
            elseif sum(recombinant_mask) == 1
                #Assign the 1 matched alleles recombinant r
                recombination_vec[recombinant_mask] = collect(recombination_vec[recombinant_mask]+[r])
        end
        recombination_matrix[i,:]=recombination_vec
    end
    return(recombination_matrix)
end


Build_recombination_matrix

## Simulation

In [19]:
"""
Defines starting population and matrices, 
runs simulation and 
calculates additional metrics from output.
"""
function Simulation_wrapper(params,             #Dictionary of parameters
                            A_alleles_content,  #Dictionary of components in each A allele
                            A_alleles_ir,       #Dictionary of disruption type in each A allele
                            B_alleles_content,  #Dictionary of components in each B llele
                            B_alleles_ir,       #Dictionary of disruption type in each B allele
                            fitness_effect,     #adult or larvae
                            transgenic_genotype,#e.g. ["A1" "A2"; "B1" "B2"]
                            release_freq,       #Frequency of released males (as)
                            resistant_allele, #e.g. "A2"
                            resistant_freq,     #Frequency of resistant allele
                            t)                  #number of generations
    
    #Define lists of alleles and genotypes based on the number of A and B alleles
    A_alleles,
    B_alleles,
    alleles,
    A_genotypes,
    B_genotypes,
    genotypes = define_string_genotypes(length(A_alleles_content),length(B_alleles_content));
    
    #Initiate the population
    starting_population_post_release = release_transgenic_males(
        Initiate_prerelease_population_3lifestages(resistant_allele, resistant_freq, params, genotypes),
        transgenic_genotype,
        release_freq);
    
    #Define the matrices
    matrices = make_matrices(params,
                        genotypes,
                        alleles,
                        A_alleles_content,
                        A_alleles_ir,
                        A_genotypes,
                        B_alleles_content,
                        B_alleles_ir,
                        B_genotypes);
    
    #Run the simulation
    output = Simulate_timeseries(
                    params,
                    matrices,
                    fitness_effect,
                    t,
                    starting_population_post_release,
                    genotypes,
                    alleles)

    data_dict = Simulation_processing_wrapper(output,A_alleles,B_alleles,alleles,genotypes)
    
    return(data_dict)
end

Simulation_wrapper


###### Defining the population upon release

We begin with a Nx6 matrix of numbers of individuals of each genotype after transgenic release, where N is the number of genotypes. Each row refers to a genotype and each column to the following catagories or organism: Female zygotes, males zygotes, female pupae, male pupae, female adults and male adults.
The pre-release population may contain only WT individuals (ABAB) or may contain a resistant allele, assumed to be at Hardy-Weinberg equilibrium at a user-define allele frequency. 

In [20]:
"""
Initiates a dictionary of genotypes frequencies, where each genotype freq has an initial value of 0. 
"""
function Initiate_genotype_freq(genotypes)
    freq_genotype = Dict()
    for i in 1:length(genotypes)
       freq_genotype[genotypes[i]] = 0.0
    end
    return(freq_genotype)
end

"""
Returns 2 dictionaries containing the numbers of each genotype, one for females and one for males.
The total number of males or females is defined by the variable params: OJ, f and α and refers to the equilibrium population size of each population.
The population can contain both WT individuals (ABAB) and genotypes containing a resistant allele.
The population is assumed to be at hardy weinberg equilibrium, with the resistant allele at a frequency defined by the variable "resistant_freq".
"""
function Initiate_prerelease_population(resistant_allele, resistant_freq,params,genotypes)
    
    Rm = (params["f"]*params["OJ"])/2
    Nf = params["α"]*(Rm-1)/params["f"]

    #Define each frequency initially as 0
    genotype_freqf = Initiate_genotype_freq(genotypes)
    genotype_freqm = Initiate_genotype_freq(genotypes)

    #homozygous genotype
    if resistant_allele[1] == 'A'
        het = ["A1" resistant_allele; "B1" "B1"]
        hom = [resistant_allele resistant_allele; "B1" "B1"]
    elseif resistant_allele[1] == 'B'
        het = ["A1" "A1"; "B1" resistant_allele]
        hom = ["A1" "A1"; resistant_allele resistant_allele]
    end
    
    genotype_freqf[["A1" "A1"; "B1" "B1"]] = (1-resistant_freq)^2*Nf
    genotype_freqf[hom] = resistant_freq^2*Nf
    genotype_freqf[het] = 2 * resistant_freq * (1-resistant_freq)*Nf

    genotype_freqm[["A1" "A1"; "B1" "B1"]] = (1-resistant_freq)^2*Nf
    genotype_freqm[hom] = resistant_freq^2*Nf
    genotype_freqm[het] = 2 * resistant_freq * (1-resistant_freq)*Nf

    return(genotype_freqf,genotype_freqm)
end


"""
Initiates the prerelease population of adults. 
Back-calculates the number of zygotes and pupae for a given adult population and parameter values. 
Returns a 6xN matrix, where N is the number of genotypes, of numbers of zygote, pupae and adults for females and males, with column headers labelled by genotype.
"""
function Initiate_prerelease_population_3lifestages(resistant_allele, resistant_freq, params, genotypes)
    
    genotype_Nf,genotype_Nm = Initiate_prerelease_population(resistant_allele, resistant_freq, params, genotypes)

    starting_population = Array{Float64}(undef,(6,length(genotypes)))
 
    #Modify data type to arrays in the the order of genotypes
    starting_population[5,:] = [genotype_Nf[i] for i in genotypes]
    starting_population[6,:] = [genotype_Nm[i] for i in genotypes]
    
    #Assuming the population is at equilibrium AND the population is pre-release
    #No. zygotes
    starting_population[1,:] =  starting_population[5,:] .* params["f"] /2
    starting_population[2,:] =  starting_population[6,:] .* params["f"] /2

    #No. zygotes after density dependent mortality (pupae)
    n_zygotes_sum = sum(starting_population[1,:])+sum(starting_population[2,:])
    juvenile_survival_prob = params["OJ"]*(params["α"]/(params["α"]+n_zygotes_sum))
    starting_population[3,:]= starting_population[5,:] .* params["f"] /2 * juvenile_survival_prob
    starting_population[4,:] = starting_population[6,:] .* params["f"] /2 * juvenile_survival_prob
    genotypes_sym = [join(t) for t in genotypes]
    N_individuals = DataFrame(starting_population,Symbol.(genotypes_sym))
    
    return(N_individuals)
end

"""
Takes in a 6xN matrix, where N is the number of genotypes, defining the numbers of genotypes for a population pre-release and 
returns a Nx6 matrix of numbers of each genotype after release of males. 
The frequency and genotype of the release males is defined by the input.
"""
function release_transgenic_males(starting_population,transgenic_genotype,release_freq)
    #Here we make the assumption that all released individuals are fully fit
    transgenic_genotype = Symbol(join(transgenic_genotype))
    starting_population[6,transgenic_genotype] = sum(starting_population[6,:])*release_freq
    
    starting_population = transpose(Array(starting_population))
    
    return(starting_population)
end


release_transgenic_males


###### Time series simulation

Parameter values and the allele content (transcriptional units) and fitness type for each locus are used to produce matrices (using the functions in the Model Definition section) for applying fitness, homing, sex ratio and recombination processes to the population.

During each generation:
* Eggs and sperm are produced, taking into consideration homing, recombination and sex ratio bias. 
* Zygotes are procuded assuming random mating and that males are not limiting. 
* Fitness effects are applied at either the zygote or adult stage. 
* Density dependence is applied at the zygote stage after fitness effects (if applicable to zygotes). 
* Censusing of both the zygote and adults are taken after any fitness effects are applied.

The ouput is a dictionary of time-series for allele or genotype frequencies, numbers of individuals and correlation between constructs.


In [22]:
"""
Builds the matrices to be used based on the 
parameters, 
construct contents, 
fitness types
and numbers of alleles/genotypes. 
"""
function make_matrices(params,            #Dictionary of parameters
                        genotypes,        #List of genotypes, each entry = ["A1" "A2"; "B1" "B2"]
                        alleles,          #List of alleles, each entry = ["A1", "B1"]
                        A_alleles_content,#Dictionary of components in each A allele
                        A_alleles_ir,     #Dictionary of disruption type in each A allele
                        A_genotypes,      #List of A genotypes, each entry = ["A1", "A1"]
                        B_alleles_content,#Dictionary of components in each B allele
                        B_alleles_ir,     #Dictionary of disruption type in each B allele
                        B_genotypes)      #List of B genotypes, each entry = ["B1", "B1"]
    
    A_alleles = keys(A_alleles_content)
    B_alleles = keys(B_alleles_content)
    
    fitness_matrix = Build_fitness_matrix(A_alleles_content,A_alleles_ir,A_genotypes,B_alleles_content,B_alleles_ir,B_genotypes,genotypes,params)
    homing_matrix_f = Build_homing_matrix(A_alleles,A_alleles_content,A_alleles_ir,B_alleles,B_alleles_content,B_alleles_ir,genotypes,params["caf"],params["cbf"],params["jaf"],params["jbf"],params["paf"],params["pbf"],params["daf"],params["dbf"]);
    homing_matrix_m = Build_homing_matrix(A_alleles,A_alleles_content,A_alleles_ir,B_alleles,B_alleles_content,B_alleles_ir,genotypes,params["cam"],params["cbm"],params["jam"],params["jbm"],params["pam"],params["pbm"],params["dam"],params["dbm"]);
    sex_ratio_matrix = Build_sex_ratio_matrix(A_alleles,A_alleles_content,B_alleles,B_alleles_content,genotypes,params)
    recombination_matrix = Build_recombination_matrix(genotypes,alleles,params);

    return(fitness_matrix,
            homing_matrix_f,
            homing_matrix_m,
            recombination_matrix,
            sex_ratio_matrix)
end

"""
Takes in the number of females adults, female homing matrix, recombination matrix and parameter values. 
The number of eggs produced by each genotype is calculated. 
Homing is applied by multiplying the vector of egg numbers per genotype by the female homing matrix, 
and summing over the columns. 
Recomnbination is applied and the number of each gamete produced per genotype is calculated (45x9 matrix) 
by multiplyng the number of eggs per genotype after homing by the recombination matrix. 
The matrix is summed over each column to find the number of eggs produced of each gamete genotype, 
returning a vector with a length equal to the number of gametes.
"""
function make_eggs(genotype_Nf_adult,          #list of numbers of adult females of each genotype
                    homing_matrix_f_vals,      #female homing matrix
                    recombination_matrix_vals, #recombination matrix
                    params)                    #Dictionary of parameters

                #Calulate the number of eggs produced by each reproducing adult female
                neggs_per_genotype = genotype_Nf_adult * params["f"];

                #Homing:
                #Calculate the numbers of eggs produced from each genotype after homing
                neggs_per_genotype_per_genotype_homing = neggs_per_genotype.*homing_matrix_f_vals
                neggs_per_genotype_homing = sum(neggs_per_genotype_per_genotype_homing,dims=1)[:]

                #recombination
                #Calulate the number of each gamete produced by each genotype after recombination
                neggs_per_genotype_per_gamete = neggs_per_genotype_homing .*recombination_matrix_vals;
                # Calculate the number of each gamate produced by the female population in total
                neggs_per_gamete = sum(neggs_per_genotype_per_gamete,dims=1)[:];
end

"""
Takes in the number of male adults, male homing matrix, recombination matrix and sex ratio matrix. 
Males are assumed to be not limiting, so the number of males is converted to frequency. 
Homing is applied by multiplying the frequency of males per genotype by the male homing matrix, 
and summing over the columns. 
Recomnbination is applied and the frequency of each gamete produced per genotype is calculated (45x9 matrix) 
by multiplyng the frequency of males per genotype after homing by the recombination matrix. 
Sex ratio bias is applied by multiplying the 45x9 matrix containing the frequency of each gamete per genotype
by each column of the the sex ratio matrix separately, providing 2 matrices, one for the proportion of each Y-bearing gamete
genotype per adult male and another for the X-bearing equivalent.
The columns of each X-bearing and Y-bearing sperm matrix are summed to find the frequency of 
each gamete genotype produced.
Returns a Gx2 matrix, where G is the number of gametes, containing the frequency of X-bearing sperm (column 1) and Y-bearing sperm (column 2)
of each gamate genotype (rows).
"""
function make_sperm(genotype_Nm_adult,        #list of numbers of adult males of each genotype
                    homing_matrix_m_vals,     #male homing matrix
                    recombination_matrix_vals,#recombination matrix
                    sex_ratio_matrix_vals)    #sex ration matrix

                #conver male numbers to frequency. 
                genotype_freqm_fitness = genotype_Nm_adult./sum(genotype_Nm_adult)

                #Homing:
                genotype_freqm_per_genotype_homing = genotype_freqm_fitness.*homing_matrix_m_vals
                genotype_freqm_homing = sum(genotype_freqm_per_genotype_homing,dims=1)[:]

                #recombination
                freqsperm_per_genotype = genotype_freqm_homing.*recombination_matrix_vals;

                #sex ratio
                freqsperm_per_genotype_per_gamete_X = freqsperm_per_genotype.*sex_ratio_matrix_vals[:,1]
                freqsperm_per_genotype_per_gamete_Y = freqsperm_per_genotype.*sex_ratio_matrix_vals[:,2]

                freqsperm_per_gamete_X = sum(freqsperm_per_genotype_per_gamete_X,dims=1)[:]
                freqsperm_per_gamete_Y = sum(freqsperm_per_genotype_per_gamete_Y,dims=1)[:]
                freqsperm_per_gamete = vcat(freqsperm_per_gamete_X,freqsperm_per_gamete_Y)

                return(hcat(freqsperm_per_gamete_X,freqsperm_per_gamete_Y))
end

"""
Takes in a Gx3 matrix, where G is the number of gametes, containing egg and sperm (X and Y) 
numbers or frequencies and calculates the proportion of 
each female and male zygote produced of each diploid genotype. 
Assumes random mating and that males are not limiting. 
Returns as Nx2 matrix, where N is the number of diploid genotypes, containing numbers of female (column 1) and male (column 2) genotypes. 
"""
function Find_nzygotes(gametes,   #matrix containing number of eggs, X-bearing sperm and Y-bearing sperm of each gamete genotype
                        alleles,  #list of alleles, each entry = ["A1", "B1"]
                        genotypes)#list of genotypes, each entry = ["A1" "A2"; "B1" "B2"]
    
    #Taking in matrices of the number of eggs and sperm produced by each genotype:
    #Calculates the proportion of each female and male zygote of each genotype produced.
    
    neggs_genotype = gametes[:,1]
    freqsperm_genotype_X = gametes[:,2]
    freqsperm_genotype_Y = gametes[:,3]
    
    #Female zygote genotype 
    genotype_freq_F_vec = []
    #Male zygote genotype 
    genotype_freq_M_vec = []
    
    #For each potential zygote genotype
    for genotype in genotypes
        #Check if homozygote
        if genotype[:,1] == genotype[:,2]
            #Find the index of the gamete present in the genotype
            index = findall(x->x==genotype[:,1],alleles)[1]
            #multiply the number of eggs by the frequency X sperm of the specified gamete
            genotype_freq_F = neggs_genotype[index]*freqsperm_genotype_X[index]
            #multiply the number of eggs by the frequency Y sperm of the specified gamete
            genotype_freq_M = neggs_genotype[index]*freqsperm_genotype_Y[index]
        else
            #Find the index of each allele in the genotype
            index1 = findall(x->x==genotype[:,1],alleles)[1]
            index2 = findall(x->x==genotype[:,2],alleles)[1]
            #multiply the number of eggs by the frequency X sperm of the specified gametes in both combinations
            genotype_freq_F = ((neggs_genotype[index1]*freqsperm_genotype_X[index2])) + ((freqsperm_genotype_X[index1]*neggs_genotype[index2]))
            #multiply the number of eggs by the frequency Y sperm of the specified gametes in both combinations
            genotype_freq_M = ((neggs_genotype[index1]*freqsperm_genotype_Y[index2])) + ((freqsperm_genotype_Y[index1]*neggs_genotype[index2]))

        end
        push!(genotype_freq_F_vec,genotype_freq_F)
        push!(genotype_freq_M_vec,genotype_freq_M)
    end
    
    
    return(hcat(genotype_freq_F_vec,genotype_freq_M_vec))
end

"""
Takes in a set of parameters and a Nx2 matrix, where N is the number of diploid genotypes, of numbers per genotype. 
Applies the density dependent function to the population. 
Returns a Nx2 matrix of numbers of individuals per genotype after density dependence has been applied. 
"""
function apply_density_dependence(params, #Dictionary of parameters
                                n_zygotes)#matrix containing male and female zygote numbers for each genotype
    n_zygotes_sum = sum(n_zygotes)
    juvenile_survival_prob = params["OJ"]*(params["α"]/(params["α"]+n_zygotes_sum))
    n_zygote_dd = n_zygotes * juvenile_survival_prob

    return(n_zygote_dd)
end

"""
Simulation of population through time, in discrete non-overlapping generations. 
The function takes in: parameters, matrixes, fitness effect, a starting population (post release) and 
the number of generations required.
Eggs and sperm are produced, taking into consideration homing, recombination and sex ratio bias. 
Zygotes are procuded assumin random mating and that males are not limiting. 
fitness effects are applied according to the fitness effect input(zygote or adult)
Density dependence is applied at the zygote stage before zygotes develop into pupae. 
Zygote mortality (if applied) is applied before density dependence and the zygote population is censused after. 
The adult population is also censused after fitness effects are applied (if applicable).
Zygote, pupae and adult population numbers for each sex are saved in a 45x6xt matrix, 
and egg numbers and sperm frequencies are saved in a 9x3xt matrix, where t is the number of generations. 
To enable intuitive data access, the output matrices are processed into a dictionary of dataframes
containing txN matrices for each diploid population, a txG matrix for egg numbes and a tx18 for sperm frequency,
where N is the number of diploid genotypes and G is the number of gametes,
where columns are accessible by labelled genotypes.
"""
function Simulate_timeseries(params,        #Dictionary of parameters
                                        matrices,      #list of matrices
                                        fitness_effect,#adult or larvae
                                        t,             #number of generations
                                        starting_population_post_release,#starting population matrix
                                        genotypes,     #list of genotypes, each entry = ["A1" "A2"; "B1" "B2"]
                                        alleles)       #list of alleles, each entry = ["A1", "B1"]
    
    #Unpack list of pre-defined matrices
    fitness_matrix_vals,
    homing_matrix_m_vals,
    homing_matrix_f_vals,
    recombination_matrix_vals,
    sex_ratio_matrix_vals = matrices
    
    #define (45x6xt) storage for n. individuals (f zygote, m zygote, f pupae, m pupae, f adult, m adult)
    n = Array{Float64}(undef,(length(genotypes),6,t+1))
    #define (9x3xt) storage for n. gametes (eggs, X-bearing sperm, Y-bearing sperm))
    g = Array{Float64}(undef,(length(alleles),3,t+1))

    #save starting population from input
    n[:,:,1]=starting_population_post_release 
    
    #same no. adults in starting pop
    genotype_Nf_adult=n[:,5,1]
    genotype_Nm_adult=n[:,6,1]

    for gen in 1:t
        
        #female adults produce eggs 
        neggs_per_gamete =  make_eggs(genotype_Nf_adult,homing_matrix_f_vals,recombination_matrix_vals,params)
        g[:,1,gen+1] = neggs_per_gamete
        #male adults produce sperm
        freqsperm_per_gamete = make_sperm(genotype_Nm_adult,homing_matrix_m_vals,recombination_matrix_vals,sex_ratio_matrix_vals)
        g[:,2:3,gen+1] = freqsperm_per_gamete
        #zygotes are generated from eggs and sperm
        n_zygotes = Find_nzygotes(g[:,:,gen+1],alleles,genotypes)
        #fitess costs are applied to zygotes if applicable
        fitness_effect == "zygote" ? n_zygotes = n_zygotes .*fitness_matrix_vals : n_zygotes = n_zygotes
        n[:,1:2,gen+1] = n_zygotes
        #density dependent mortality is applied
        n_pupae = apply_density_dependence(params,n_zygotes)
        n[:,3:4,gen+1] = n_pupae
        #fitess costs are applied to zygotes if applicable
        fitness_effect == "adult" ? n_adults = n_pupae .*fitness_matrix_vals : n_adults = n_pupae
        n[:,5:6,gen+1] = n_adults
        #Number of adults are updated for generation t+1
        genotype_Nf_adult=n_adults[:,1]
        genotype_Nm_adult=n_adults[:,2]
    end 
    
    #Store as dataframes in dictionary for intuitive labelled access 
    genotypes_sym = [join(t) for t in genotypes]
    alleles_sym = [join(t) for t in alleles]
    header = Symbol.(genotypes_sym)
    output = Dict(
        "genotypenumber_adult_female" => DataFrame(transpose(n[:,5,:]), header),
        "genotypenumber_adult_male" => DataFrame(transpose(n[:,6,:]), header),
        "genotypenumber_pupae_female" => DataFrame(transpose(n[:,3,:]), header),
        "genotypenumber_pupae_male" => DataFrame(transpose(n[:,4,:]), header),
        "genotypenumber_zygote_female" => DataFrame(transpose(n[:,1,:]), header),
        "genotypenumber_zygote_male" => DataFrame(transpose(n[:,2,:]), header),
        "eggnumber" => DataFrame(transpose(g[:,1,:]), Symbol.(alleles_sym)),
        "spermfreq" => DataFrame(hcat(transpose(g[:,2,:]),transpose(g[:,3,:])),Symbol.([string.(alleles_sym).*"X" ; string.(alleles_sym).*"Y"]))
    )


    return(output)
end

Make_dataframe (generic function with 1 method)


###### Post simulation processing

In [24]:
"""
Takes in a simulation output and calculates various summary metrics from the numbers of individuals/gametes. 
Returns a dictionary containing the same output data with additional entries. 
"""
function Simulation_processing_wrapper(output,A_alleles,B_alleles,alleles,genotypes)

    #Calculate additional values of interest from the time series output
    total_Nf = hcat(sum.(eachrow(output["genotypenumber_pupae_female"])),sum.(eachrow(output["genotypenumber_adult_female"])))
    total_Nf = DataFrame(total_Nf,["pupae","adult"])
    relative_Nf = Find_relative_Nf(total_Nf);

    #For the zygotes (regardless of reproductive fitness)
    genotype_freqzf_store = freq_from_no_df(output["genotypenumber_zygote_female"])
    genotype_freqzm_store = freq_from_no_df(output["genotypenumber_zygote_male"])
    genotype_freqz_store = (genotype_freqzf_store.+genotype_freqzm_store)./2;
    allele_freqz = Find_allele_frequencies_from_genotypes_df(genotypes,genotype_freqz_store,A_alleles,B_alleles);
    correlationz = Find_correlation_df(genotypes,alleles,genotype_freqz_store,A_alleles,B_alleles,"A2","B2")

    #For all pupae (regardless of reproductive fitness)
    genotype_freqpf_store = freq_from_no_df(output["genotypenumber_pupae_female"])
    genotype_freqpm_store = freq_from_no_df(output["genotypenumber_pupae_male"])
    genotype_freqp_store = (genotype_freqpf_store.+genotype_freqpm_store)./2;
    allele_freqp = Find_allele_frequencies_from_genotypes_df(genotypes,genotype_freqp_store,A_alleles,B_alleles);
    correlationp = Find_correlation_df(genotypes,alleles,genotype_freqp_store,A_alleles,B_alleles,"A2","B2")

    #For the reproductive adults (regardless of reproductive fitness)
    genotype_freqaf_store = freq_from_no_df(output["genotypenumber_adult_female"])
    genotype_freqam_store = freq_from_no_df(output["genotypenumber_adult_male"])
    genotype_freqa_store = (genotype_freqaf_store.+genotype_freqam_store)./2;
    allele_freqa = Find_allele_frequencies_from_genotypes_df(genotypes,genotype_freqa_store,A_alleles,B_alleles);
    correlationa = Find_correlation_df(genotypes,alleles,genotype_freqa_store,A_alleles,B_alleles,"A2","B2")

    #save in a dictionary
    data_dict = Dict(
        
        "genotypenumber_adult_female" => output["genotypenumber_adult_female"],
        "genotypenumber_adult_male" => output["genotypenumber_adult_male"],
        "genotypenumber_pupae_female" => output["genotypenumber_pupae_female"],
        "genotypenumber_pupae_male" => output["genotypenumber_pupae_male"],
        "genotypenumber_zygote_female" => output["genotypenumber_zygote_female"],
        "genotypenumber_zygote_male" => output["genotypenumber_zygote_male"],
        
        "number_females" => total_Nf,
        "relative_number_females" => relative_Nf,
        
        "genotypefreq_adult_female" => genotype_freqaf_store,
        "genotypefreq_adult_male" => genotype_freqam_store,
        "genotypefreq_adult" => genotype_freqa_store,
        
        "genotypefreq_pupae_female" => genotype_freqpf_store,
        "genotypefreq_pupae_male" => genotype_freqpm_store,
        "genotypefreq_pupae" => genotype_freqp_store,
        
        "genotypefreq_zygote_female" => genotype_freqzf_store,
        "genotypefreq_zygote_male" => genotype_freqzm_store,
        "genotypefreq_zygote" => genotype_freqz_store,
        
        "allelefreq_adult" => allele_freqa,
        "allelefreq_pupae" => allele_freqp,
        "allelefreq_zygote" => allele_freqz,
        
        "correlation_adult" => correlationa,
        "correlation_pupae" => correlationp,
        "correlation_zygote" => correlationz
    );
    return(data_dict)
end

"""Calculate the row-wise frequency"""
function freq_from_no_df(genotype_N_store)
    genotype_N_store./[sum(row) for row = eachrow(genotype_N_store)]
end

"""Calculate the fraction of females relative to generation zero"""
function Find_relative_Nf(Nf_store)
    pupae = (Nf_store.pupae)/(Nf_store.pupae[1])
    adult = (Nf_store.adult)/(Nf_store.adult[1]);
    relative_Nf = DataFrame(pupae = float.(pupae),
                    adult=float.(adult))
end

"""
Takes in the genotype frequencies of a population at a single time point and 
calculated the allele frequency.
"""
function Find_allele_frequencies_from_genotypes(genotypes,freq_genotype,A_alleles,B_alleles)
    """Extract the allele frequencies from the genotype frequencies"""
    allele_freqs = Vector{Float64}()
    for allele in vcat(A_alleles,B_alleles)
        allele_freq = 0
        for i in 1:length(genotypes)
            if (allele in genotypes[i][:,1]) & (allele in genotypes[i][:,2])
                allele_freq = allele_freq+freq_genotype[i]
            elseif (allele in genotypes[i][:,1]) | (allele in genotypes[i][:,2])
                allele_freq = allele_freq+(freq_genotype[i]/2)
            end
        end
       push!(allele_freqs,allele_freq)
    end
    return(allele_freqs)
end
function Find_allele_frequencies_from_genotypes_df(genotypes,genotype_freq_store,A_alleles,B_alleles)
    header = Symbol.(vcat(A_alleles,B_alleles))
    Make_dataframe([Find_allele_frequencies_from_genotypes(genotypes,row,A_alleles,B_alleles) for row = eachrow(genotype_freq_store)],header)
end

"""
Takes in the genotype frequencies of a population at a single time point and 
calculated the frequency of each haploid genotype (chromosome).
"""
function Find_chromosome_frequencies_from_genotypes(freq_genotype,genotypes,alleles)
    """Extract the allele frequencies from the genotype frequencies"""
    allele_freqs = []
    for allele in alleles
        allele_freq = 0
        for i in 1:length(genotypes)
            if ((allele == genotypes[i][:,1]) | (allele == genotypes[i][:,2]))
                if genotypes[i][:,1] == genotypes[i][:,2]
                    allele_freq = allele_freq+(freq_genotype[i])
                else
                    allele_freq = allele_freq+(freq_genotype[i]/ 2)
                end
            end
        end
       append!(allele_freqs,allele_freq)
    end
    return(allele_freqs)
end
function Find_chromosome_frequencies_from_genotypes_df(genotype_freq_store,genotypes,alleles)
    header = Symbol.(alleles)
    Make_dataframe([Find_chromosome_frequencies_from_genotypes(row,genotypes,alleles) for row = eachrow(genotype_freq_store)],header)
end

"""
Takes in the genotype frequencies of a population at a single time point and 
calculated the correlation bewteen the constructs a and b.
"""
function Find_correlation(genotypes,alleles,genotype_freq,A_alleles,B_alleles,A,B)
    
    chrom_freq = Find_chromosome_frequencies_from_genotypes(genotype_freq,genotypes,alleles)
    ab_freq = chrom_freq[findall(x->x==[A,B],alleles)[1]]
    
    allele_freq = Find_allele_frequencies_from_genotypes(genotypes,genotype_freq,A_alleles,B_alleles)
    AB_alleles = vcat(A_alleles,B_alleles)
    a_freq = allele_freq[findall(x->x==A,AB_alleles)[1]]
    b_freq = allele_freq[findall(x->x==B,AB_alleles)[1]]
    
    if (float(b_freq) > 0.0)
        cor = (ab_freq - (a_freq*b_freq))/sqrt(float(a_freq*(1-a_freq)*b_freq*(1-b_freq)))
    else
        cor = 0.0
    end
end
function Find_correlation_df(genotypes,alleles,genotype_freq_store,A_alleles,B_alleles,A,B)
    header = ["Correlation"]
    Make_dataframe([Find_correlation(genotypes,alleles,row,A_alleles,B_alleles,A,B) for row = eachrow(genotype_freq_store)],header)
end

"""Processes a matrix into a dataframe"""
function Make_dataframe(store, header)
    df = DataFrame(transpose(hcat(store...)))
    rename!(df, header)
end

Find_correlation_df (generic function with 1 method)

###### Input/Output functions

In [ ]:
"""
Writes output dictionary to file, with each entry of the dictionary being written to a different txt file.
"""
function Sim_IO_save(file_prefix,
                            params,
                            data_dict,
                            t)
    
    #Write to info file the parameters used to run the sumulation
    fn = file_prefix*"_info.txt"
    io = open(fn, "w");
    write(io,"sim_length \t"*string(t)*"\n")
    p_label = keys(params)
    for p in p_label
        write(io,p*"\t"*string(params[p])*"\n")
    end
    close(io)
    
    #Write to file the data according to the dictionary label
    for i in [keys(data_dict)...]
        CSV.write(file_prefix*"_"*i*".csv", data_dict[i])
    end
end

"""
Reads output files and stores in dictionary.
"""
function Sim_IO_read(file_prefix,
                               data_labels)
    data_dict = Dict()
    for label in data_labels
        data_dict[label] = CSV.read(file_prefix*"_"*label*".csv",DataFrame)
    end
    return(data_dict)
end